In [17]:
import pandas as pd
import numpy as np
from urllib.parse import urlparse, unquote
from preprocessor import classify_features

In [18]:
data = pd.read_csv("Full-Dataset/method_na_removed.csv")

In [19]:
urlData = pd.Series(
    map(urlparse, data["http.request.full_uri"])
)


In [20]:
def query_params_to_dict(series:pd.Series, to_json=False):
    import json
    return pd.Series([
        (
            json.dumps(dict((ob.split("=")[0], unquote("=".join(ob.split("=")[1:])))for ob in _.split("&")), indent=2, ensure_ascii=True) 
            if to_json else
            dict((ob.split("=")[0], unquote("=".join(ob.split("=")[1:])))for ob in _.split("&")) 
        )
            for _ in series
    ])

In [21]:
urlDatas = pd.DataFrame({
    key: [getattr(row,key) for row in urlData]
    for key in ["scheme", "netloc", "path", "params", "query", "fragment",]
})

In [22]:
data = pd.DataFrame({
    **{col: data[col] for col in data.columns if col},
    **{col: urlDatas[col] for col in urlDatas.columns},
})

In [23]:
used_col, unused_col = classify_features(data, ["Attack_type"], debug=True)

scheme : unique value column
netloc : unique value column
params : unique value column
fragment : unique value column


In [24]:
for c in unused_col:
    print(f"{c} = {data[c].unique().shape[0]}")

scheme = 1
fragment = 1
netloc = 1
params = 1


In [25]:
data = data.drop(unused_col, axis="columns")

In [26]:
data["query"] = query_params_to_dict(data["query"])

In [27]:
data.head()
# Needing RNN or CNN to process the "query" of the dataset.
# path must be deleted in training, because this dataset's uri.path has the data of "which attack has been used".

,frame.time,http.request.full_uri,tcp.checksum,tcp.options,tcp.payload,Attack_type,GET,POST,TRACE,HTTP/1.0,HTTP/1.1,path,query
0,2021 21:35:18.571627000,http://192.168.0.128/DVWA/vulnerabilities/sqli...,0x000023bd,0101080a9b1e676b9eabb8af,474554202f445657412f76756c6e65726162696c697469...,SQL_injection,True,False,False,False,True,/DVWA/vulnerabilities/sqli_blind/,"{\n ""id"": ""2"",\n ""Submit"": ""Submitid=53""\n}"
1,2021 21:35:18.642567000,http://192.168.0.128/DVWA/vulnerabilities/sqli...,0x0000d49b,0101080a9b1e67b49eabfa17,474554202f445657412f76756c6e65726162696c697469...,SQL_injection,True,False,False,False,True,/DVWA/vulnerabilities/sqli_blind/,"{\n ""id"": ""2'.\"")(,.(.("",\n ""Submit"": ""Submi..."
2,2021 21:35:18.691481000,http://192.168.0.128/DVWA/vulnerabilities/sqli...,0x00004a68,0101080a9b1e67e59eabfa5c,474554202f445657412f76756c6e65726162696c697469...,SQL_injection,True,False,False,False,True,/DVWA/vulnerabilities/sqli_blind/,"{\n ""id"": ""2'FHBrxG<'\"">BjxRQk"",\n ""Submit"":..."
3,2021 21:35:18.816771000,http://192.168.0.128/DVWA/vulnerabilities/sqli...,0x00002fad,0101080a9b1e68629eabfaac,474554202f445657412f76756c6e65726162696c697469...,SQL_injection,True,False,False,False,True,/DVWA/vulnerabilities/sqli_blind/,"{\n ""id"": ""2) AND 9522=9017 AND (5824=5824"",\..."
4,2021 21:35:18.868654000,http://192.168.0.128/DVWA/vulnerabilities/sqli...,0x000045cd,0101080a9b1e68969eabfb0f,474554202f445657412f76756c6e65726162696c697469...,SQL_injection,True,False,False,False,True,/DVWA/vulnerabilities/sqli_blind/,"{\n ""id"": ""2 AND 5107=2126"",\n ""Submit"": ""Su..."


In [28]:
data.to_csv("Full-Dataset/processed/url_encoded_dataset.csv", sep=",")